In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [2]:
park = Customer('Park', 100)
park

Customer(name='Park', fidelity=100)

In [16]:
class LineItem:
  """구매할 물품/갯수 생성해서 총 가격을 반환"""
  def __init__(self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price

  def total(self):
    return self.price * self.quantity

In [17]:
class Order:
  """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
  def __init__(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion

  def total(self):
    """__total 속성이 없으면 전체 계산할 값을 계산"""
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    """할인금액 차감"""
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion.discount(self)
    return self.total() - discount

  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())

In [18]:
class Promotion(ABC):
  """할인 혜택 클래스들의 형태를 선언"""

  @abstractmethod
  def discount(self, order):
    """할인액을 구체적인 숫자로 반환"""
    pass

In [28]:
class FidelityPromo(Promotion):
  """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""

  def discount(self, order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
  """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""

  def discount(self, order):
    discount = 0
    for item in order.cart:
      if item.quantity >= 20:
        discount += item.total() * 0.1
    return discount

class LargeOrderPromo(Promotion):
  """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""

  def discount(self, order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
      return order.total() * 0.07
    return 0

In [29]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [30]:
Order(joe, cart, FidelityPromo())


<Order total: 42.00 due: 42.00>

In [31]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [32]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [33]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [34]:
long_order = [LineItem(str(item_code), 1, 1.0)
for item_code in range(10)]

In [35]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [36]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [50]:
class Order:
  """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
  def __init__(self, customer,cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion

  def total(self):
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion(self)
    return self.total() - discount

  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())

In [51]:
def fidelity_promo(order):
  """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
  """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount

def large_order_promo(order):
  """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

In [52]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [53]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [54]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [55]:
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [56]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
  """최대로 할인받을 금액을 반환"""
  return max(promo(order) for promo in promos)

In [57]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [58]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [59]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [60]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'ABC', 'abstractmethod', 'namedtuple', 'Customer', '_i2', 'park', '_2', '_i3', 'LineItem', '_i4', 'Order', '_i5', 'Promotion', '_i6', 'FidelityPromo', 'BulkItemPromo', 'LArgeOrderPromo', '_i7', 'joe', 'ann', 'cart', '_i8', '_8', '_i9', '_9', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_15', '_i16', '_i17', '_i18', '_i19', '_i20', '_i21', '_21', '_i22', '_22', '_i23', 'banana_cart', '_i24', '_24', '_i25', 'long_order', '_i26', '_i27', '_i28', 'LargeOrderPromo', '_i29', '_i30', '_30', '_i31', '_31', '_i32', '_i33', '_33', '_i34', '_i35', '_35', '_i36', '_36', '_i37', '_i38', '_i40', '_i41', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i42', '_42', '_i43', '_43', '_i44', '_i45', '_45', '_i46', '_i47', '_i48', '_i49', '_49', '_i50', '_i51', '_i52', '_52', '

In [61]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [62]:
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]